This notebook allows you to manage web requests that invoke Flask-based HTTP functions as needed to load additional cleaned data into the knowledge base, and subsequently update the materialized view used for popularity score calculation.

In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def dispatch_etl_requests(
    start_row_index, 
    end_row_index, 
    source, 
    batch_size=20, 
    max_workers=10, 
    base_url="http://127.0.0.1:8181/etl"
):
    """
    Dispatches concurrent ETL requests to the specified endpoint.

    Args:
        start_row_index (int): Start index (inclusive)
        end_row_index (int): End index (inclusive)
        source (str): Specifies the data source.
            - 'training': Uses the knowledge base or an external database.
            - 'testing': Uses the Heymate internal database.
        batch_size (int): Number of rows per batch
        max_workers (int): Number of concurrent workers
        base_url (str): Base URL of the ETL endpoint

    Returns:
        List of tuples: (status_code, url, response_text)
    """
    # Generate a list of URLs
    urls = [
        f"{base_url}?start_row_index={i}&end_row_index={min(i+batch_size-1, end_row_index)}&source={source}"
        for i in range(start_row_index, end_row_index + 1, batch_size)
    ]

    def fetch(url):
        try:
            response = requests.get(url)
            return response.status_code, url, response.text
        except Exception as e:
            return None, url, str(e)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch, url) for url in urls]
        for future in as_completed(futures):
            status, url, result = future.result()
            print(f"[{status}] {url} → {result[:200]}...")



In [2]:
# Example usage of dispatch_etl_requests

# This will dispatch ETL requests for rows 300101 to 300200 (inclusive),
# using the 'training' source, in batches of 30 rows each.
# Up to 16 requests will be processed concurrently.
# The requests will be sent to the specified ETL endpoint.

dispatch_etl_requests(300101, 300200, "training", batch_size=30, max_workers=16)

[200] http://127.0.0.1:8181/etl?start_row_index=300101&end_row_index=300130&source=training → ...
[200] http://127.0.0.1:8181/etl?start_row_index=300191&end_row_index=300200&source=training → ...
[200] http://127.0.0.1:8181/etl?start_row_index=300131&end_row_index=300160&source=training → ...
[200] http://127.0.0.1:8181/etl?start_row_index=300161&end_row_index=300190&source=training → ...


In [3]:
# After all new data is loaded, update the materialized view for the popularity score.
from popularity_score_calculator import calculate_and_upload
calculate_and_upload(truncate=True)

/Users/krabssiew/Desktop/MDS/Block 6/capstone/heymate-report/script/util_database_reader.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Read 30200 rows from SQL Server.
Table cleaned_menu_with_popularity is ready.
Truncating table cleaned_menu_with_popularity...
Uploaded 24407 rows to cleaned_menu_with_popularity
